# 05-Nature Tutorial- Transforming Problems

In [ ]:
#This code is at:
#https://qiskit-community.github.io/qiskit-nature/tutorials/05_problem_transformers.html

In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.problems import ElectronicBasis

driver = PySCFDriver()
driver.run_pyscf()

ao_problem = driver.to_problem(basis=ElectronicBasis.AO)
print(ao_problem.basis)

ao_hamil = ao_problem.hamiltonian
print(ao_hamil.electronic_integrals.alpha)


The provided alpha-beta overlap matrix is NOT unitary! This can happen when the alpha- and beta-spin orbitals do not span the same space. To provide an example of what this means, consider an active space chosen from unrestricted-spin orbitals. Computing <S^2> within this active space may not result in the same <S^2> value as obtained on the single-reference starting point. More importantly, this implies that the inactive subspace will account for the difference between these two <S^2> values, possibly resulting in significant spin contamination in both subspaces. You should verify whether this is intentional/acceptable or whether your choice of active space can be improved. As a reference, here is the summed-absolute deviation of `S^T @ S` from the identity: 7.064215023495638


ElectronicBasis.AO
Polynomial Tensor
 "+-":
array([[-1.12421758, -0.9652574 ],
       [-0.9652574 , -1.12421758]])
 "++--":
array([0.77460594, 0.44744572, 0.3009177 , 0.57187698, 0.44744572,
       0.77460594])


In [2]:
from qiskit_nature.second_q.formats.qcschema_translator import get_ao_to_mo_from_qcschema

qcschema = driver.to_qcschema()

basis_transformer = get_ao_to_mo_from_qcschema(qcschema)
print(basis_transformer.initial_basis)
print(basis_transformer.final_basis)

ElectronicBasis.AO
ElectronicBasis.MO


In [3]:
mo_problem = basis_transformer.transform(ao_problem)
print(mo_problem.basis)

mo_hamil = mo_problem.hamiltonian
print(mo_hamil.electronic_integrals.alpha)

ElectronicBasis.MO
Polynomial Tensor
 "+-":
array([[-1.25633907e+00,  0.00000000e+00],
       [-4.16333634e-17, -4.71896007e-01]])
 "++--":
array([[[[6.75710155e-01, 1.11022302e-16],
         [8.32667268e-17, 6.64581730e-01]],

        [[6.93889390e-17, 1.80931200e-01],
         [1.80931200e-01, 1.66533454e-16]]],


       [[[5.55111512e-17, 1.80931200e-01],
         [1.80931200e-01, 2.22044605e-16]],

        [[6.64581730e-01, 2.22044605e-16],
         [2.49800181e-16, 6.98573723e-01]]]])


In [4]:
import numpy as np
from qiskit_nature.second_q.operators import ElectronicIntegrals
from qiskit_nature.second_q.problems import ElectronicBasis
from qiskit_nature.second_q.transformers import BasisTransformer

ao2mo_alpha = np.random.random((2, 2))
ao2mo_beta = np.random.random((2, 2))

basis_transformer = BasisTransformer(
    ElectronicBasis.AO,
    ElectronicBasis.MO,
    ElectronicIntegrals.from_raw_integrals(ao2mo_alpha, h1_b=ao2mo_beta),
)


In [5]:
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(atom="Li 0 0 0; H 0 0 1.5")

full_problem = driver.run()
print(full_problem.molecule)
print(full_problem.num_particles)
print(full_problem.num_spatial_orbitals)

Molecule:
	Multiplicity: 1
	Charge: 0
	Unit: Bohr
	Geometry:
		Li	(0.0, 0.0, 0.0)
		H	(0.0, 0.0, 2.8345891868475928)
	Masses:
		Li	7
		H	1
(2, 2)
6


In [6]:
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

fc_transformer = FreezeCoreTransformer()

fc_problem = fc_transformer.transform(full_problem)
print(fc_problem.num_particles)
print(fc_problem.num_spatial_orbitals)


(1, 1)
5


In [7]:
print(fc_problem.hamiltonian.constants)



{'nuclear_repulsion_energy': 1.05835442184, 'FreezeCoreTransformer': -7.84030604879426}


In [8]:
fc_transformer = FreezeCoreTransformer(remove_orbitals=[4, 5])

fc_problem = fc_transformer.transform(full_problem)
print(fc_problem.num_particles)
print(fc_problem.num_spatial_orbitals)

(1, 1)
3


In [9]:
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(atom="Li 0 0 0; H 0 0 1.5")

full_problem = driver.run()
print(full_problem.num_particles)
print(full_problem.num_spatial_orbitals)

(2, 2)
6


In [10]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

as_transformer = ActiveSpaceTransformer(2, 2)

as_problem = as_transformer.transform(full_problem)
print(as_problem.num_particles)
print(as_problem.num_spatial_orbitals)
print(as_problem.hamiltonian.electronic_integrals.alpha)

(1, 1)
2
Polynomial Tensor
 "+-":
array([[-0.78784474,  0.0469345 ],
       [ 0.0469345 , -0.36211749]])
 "++--":
array([[[[ 0.49428349, -0.0469345 ],
         [-0.0469345 ,  0.22662427]],

        [[-0.0469345 ,  0.01213863],
         [ 0.01213863,  0.00616268]]],


       [[[-0.0469345 ,  0.01213863],
         [ 0.01213863,  0.00616268]],

        [[ 0.22662427,  0.00616268],
         [ 0.00616268,  0.33881567]]]])


In [11]:
as_transformer = ActiveSpaceTransformer(2, 2, active_orbitals=[0, 4])

as_problem = as_transformer.transform(full_problem)
print(as_problem.num_particles)
print(as_problem.num_spatial_orbitals)
print(as_problem.hamiltonian.electronic_integrals.alpha)

(1, 1)
2
Polynomial Tensor
 "+-":
array([[-4.00500243e+00, -2.58340357e-17],
       [-2.58340357e-17, -6.19047188e-01]])
 "++--":
array([[[[ 1.65816678e+00,  2.50608049e-17],
         [ 2.50608049e-17,  3.96308164e-01]],

        [[ 2.50608049e-17,  9.81922731e-03],
         [ 9.81922731e-03, -3.27279856e-18]]],


       [[[ 2.50608049e-17,  9.81922731e-03],
         [ 9.81922731e-03, -3.27279856e-18]],

        [[ 3.96308164e-01, -3.27279856e-18],
         [-3.27279856e-18,  3.12945511e-01]]]])
